In [147]:
import os
import json
import pandas as pd
import traceback
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain import HuggingFaceHub
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
import os
from dotenv import load_dotenv

load_dotenv()
open_ai_api = os.getenv("OPENAI_API_KEY")
huggingface_api = os.getenv("HUGGING_FACE_KEY")

In [148]:

llm=ChatOpenAI(openai_api_key=open_ai_api,model_name="gpt-3.5-turbo", temperature=0.5)

In [149]:
# llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64},huggingfacehub_api_token=huggingface_api)

Llama_model=Ollama(model="llama2")

In [150]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [151]:
TEMPLATE = """
Text: {text}

You are an expert MCQ maker. Based on the above text, generate a quiz with {number} multiple-choice questions for {subject} students in a {tone} tone.

Instructions:
- Ensure the questions are diverse and not repetitive.
- All questions must be strictly relevant to the provided text.
- Each question must have four options (A, B, C, D) with one correct answer.
- Output the response in valid JSON format, strictly following the example.

### RESPONSE_JSON
{response_json}
"""

In [152]:

quiz_prompt_template=PromptTemplate(input_variables=["text","number","subject","tone","response_json"],template=TEMPLATE)

In [153]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt_template,output_key="quiz",verbose=True)

In [154]:
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a multiple-choice quiz for {subject} students, 
your task is to analyze its complexity and improve it if necessary.

### Task:
- Evaluate the cognitive and analytical complexity of the quiz.
- Provide a brief **complexity analysis** (maximum 50 words).
- If any question does not match the students' abilities, **update** it.
- Adjust the **tone** and **wording** to ensure it fits the students’ cognitive level.

### Quiz_MCQs:
{quiz}

### Expert Review:
"""

In [155]:
quiz_eval_template=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)


In [156]:
eval_chain=LLMChain(llm=Llama_model,prompt=quiz_eval_template,output_key="review",verbose=True)

In [157]:
generate_quiz_chain=SequentialChain(chains=[quiz_chain,eval_chain],input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"],verbose=True)

In [158]:
file_path='../data/speech.txt'

In [159]:
file_path

'../data/speech.txt'

In [160]:

with open(file_path, 'r') as file:
    TEXT = file.read()

In [161]:
print(TEXT)

The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.

…

It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or wit

In [162]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [163]:
NUMBER=5 
SUBJECT="Democracy"
TONE="simple"

In [164]:
with get_openai_callback() as cb:
    response=generate_quiz_chain(
{
        "text": TEXT,
        "number": NUMBER,
        "subject":SUBJECT,
        "tone": TONE,
        "response_json": RESPONSE_JSON
}
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.

…

It will be all the easier for us to conduct ourselves as belligeren

In [165]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1404
Prompt Tokens:995
Completion Tokens:409
Total Cost:0.0023105


In [166]:
response

{'text': 'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward

In [167]:
quiz=response.get("quiz")

In [169]:
if not quiz.strip():  # Check if the quiz is empty
    raise ValueError("Error: The quiz output is empty!")

quiz = json.loads(quiz)  # Now safely parse

In [170]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [171]:
quiz_table_data

[{'MCQ': 'What is the main goal mentioned in the text?',
  'Choices': 'a: Conquest and dominion | b: Material compensation | c: Making the world safe for democracy | d: Seeking selfish ends',
  'Correct': 'c'},
 {'MCQ': 'How does the text describe the conduct of operations as belligerents?',
  'Choices': 'a: With animus and enmity | b: With passion and selfish object | c: Without passion and with right and fairness | d: Without regard for principles of right and fair play',
  'Correct': 'c'},
 {'MCQ': 'What is the attitude of the United States towards the German people?',
  'Choices': 'a: Enmity and hostility | b: Sincere friendship | c: Indifference | d: Conquest and dominion',
  'Correct': 'b'},
 {'MCQ': 'What principles does the text mention fighting for?',
  'Choices': 'a: Material compensation and conquest | b: Democracy and right of self-government | c: Enmity and animus | d: Dominion and indemnities',
  'Correct': 'b'},
 {'MCQ': 'What is considered more precious than peace accor

In [173]:
quiz=pd.DataFrame(quiz_table_data)
quiz.to_csv("../data/machinelearning.csv",index=False)